In [81]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
import librosa
import librosa.display
import IPython.display as ipd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
warnings.filterwarnings('ignore')


In [82]:
def mfcc_feature_extractor(audio, sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T, axis=0)

    return mfccsScaledFeatures


In [83]:
def contrast_feature_extractor(audio, sampleRate):
    stft = np.abs(librosa.stft(audio))
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate)
    contrastScaled = np.mean(contrast.T, axis=0)

    return contrastScaled


In [84]:
def tonnetz_feature_extractor(audio, sampleRate):
    tonnetz = librosa.feature.tonnetz(
        y=librosa.effects.harmonic(audio), sr=sampleRate)
    tonnetzScaled = np.mean(tonnetz.T, axis=0)

    return tonnetzScaled


In [85]:
def chroma_feature_extractor(audio,sampleRate):
    stft = np.abs(librosa.stft(audio))
    chroma = librosa.feature.chroma_stft(S=stft,sr=sampleRate)
    chromaScaled = np.mean(chroma.T,axis=0)
    
    return chromaScaled

In [86]:
def features_extractor(file):
    features=[]
    audio, sampleRate = librosa.load(file, res_type='kaiser_fast') 
    mfcc=mfcc_feature_extractor(audio,sampleRate)
    contrast = contrast_feature_extractor(audio,sampleRate)
    tonnetz = tonnetz_feature_extractor(audio,sampleRate)
    chroma = chroma_feature_extractor(audio,sampleRate)

    features.append([mfcc,contrast,tonnetz,chroma])
    features[0] = np.concatenate((features[0][0],features[0][1],features[0][2],features[0][3]))
    return features

In [87]:
def add_features(extractedFeatures,dirPath,label):
    for file in os.listdir(dirPath):
        filePath = dirPath+'/'+file
        finalClassLabels=label
        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc=mfcc_feature_extractor(audio,sampleRate)
        contrast = contrast_feature_extractor(audio,sampleRate)
        tonnetz = tonnetz_feature_extractor(audio,sampleRate)
        chroma = chroma_feature_extractor(audio,sampleRate)
        extractedFeatures.append([mfcc,contrast,tonnetz,chroma,finalClassLabels])

In [88]:
dict = {"open the door":["C:/Users/EXTRA/DSP_Task3/processing/anwar",
"C:/Users/EXTRA/DSP_Task3/processing/aya",
"C:/Users/EXTRA/DSP_Task3/processing/ehab",
"C:/Users/EXTRA/DSP_Task3/processing/zeyad"],
"others":["C:/Users/EXTRA/DSP_Task3/processing/words/Anwar",
"C:/Users/EXTRA/DSP_Task3/processing/words/Aya",
"C:/Users/EXTRA/DSP_Task3/processing/words/Ehab",
"C:/Users/EXTRA/DSP_Task3/processing/words/Zeyad"]}


In [89]:
extractedFeatures = []
for i in dict.keys():
    for x in dict[i]:
        add_features(extractedFeatures,x,i)

In [90]:
data=pd.DataFrame(extractedFeatures,columns=['mfcc','contrast','tonnetz','chroma','class'])
print(data.shape)
data.describe().T


(807, 5)


,count,unique,top,freq
mfcc,807,807,"[-358.41626, 144.1046, -4.3864827, 17.072119, ...",1
contrast,807,807,"[18.94650314889134, 16.090433605334066, 20.062...",1
tonnetz,807,807,"[0.00294895926736136, 0.022882545163613432, 0....",1
chroma,807,807,"[0.4624628, 0.5353802, 0.56193304, 0.61254495,...",1
class,807,2,others,411


In [91]:
features = data.iloc[:, 0:-1]
target = data['class']
features.tail()


,mfcc,contrast,tonnetz,chroma
802,"[-403.88452, 100.31129, 12.981465, 7.540295, 4...","[18.54460786172384, 15.118820966669379, 20.905...","[0.03725058376652236, 0.020743784015899776, 0....","[0.53858376, 0.5033339, 0.4697587, 0.41963324,..."
803,"[-394.7898, 95.47343, 18.773857, 13.397423, 1....","[18.5922662615676, 15.69210975263843, 20.65567...","[0.04421657448567877, 0.056314648814841974, 0....","[0.64130944, 0.54192483, 0.44760096, 0.4129084..."
804,"[-395.23132, 95.21313, 24.915659, 11.414965, 6...","[19.597071542742974, 15.07755072043609, 18.950...","[0.08172407391847117, 0.011595188249279607, 0....","[0.6179552, 0.530558, 0.5153319, 0.4904323, 0...."
805,"[-407.15182, 80.67283, 25.647009, 16.32321, 7....","[19.328894678503787, 15.005299721334753, 18.62...","[0.03602475840749593, 0.0644023130509403, -0.0...","[0.6979088, 0.66097105, 0.58876884, 0.5688891,..."
806,"[-425.70676, 78.90517, 27.258017, 16.093987, 1...","[18.075636513475803, 14.369919534135114, 17.71...","[0.06542340934393284, 0.05735679501281795, 0.0...","[0.6332846, 0.6711462, 0.6781552, 0.6142667, 0..."


In [92]:
features = features.values.tolist()
for i in range(len(features)):
    features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))

In [93]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)


In [94]:
xTrain, xTest, yTrain, yTest = train_test_split(features, target, test_size=0.25, random_state=105)


In [95]:
classifier = SVC(kernel='linear', decision_function_shape="ovo")
classifier.fit(xTrain,yTrain)

SVC(decision_function_shape='ovo', kernel='linear')

In [96]:
preds = classifier.predict(xTest)


In [97]:
def prediction(pred):
    if pred == 0:
        print("Correct!")
    elif pred == 1:
        print("Wrong password")
  


In [98]:
print(classification_report(yTest,preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       100
           1       0.99      0.99      0.99       102

    accuracy                           0.99       202
   macro avg       0.99      0.99      0.99       202
weighted avg       0.99      0.99      0.99       202



In [99]:
testFeatures = features_extractor(
    "C:/Users/EXTRA/DSP_Task3/processing/ehab/voice1619028.wav")
pred = classifier.predict(testFeatures)
prediction(pred)
testFeatures = features_extractor(
    "C:/Users/EXTRA/DSP_Task3/processing/words/Anwar/voice28110762.wav")
pred = classifier.predict(testFeatures)
prediction(pred)


Correct!
Wrong password


In [100]:
pickle_out = open("password_model.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()
